# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
DATA_FOLDER = '/home/robin/GIT/ADA2017-Tutorials/02 - Intro to Pandas/Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.
print(DATA_FOLDER)

/home/robin/GIT/ADA2017-Tutorials/02 - Intro to Pandas/Data


In [2]:
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [3]:
import calendar
import glob
from datetime import datetime

In [4]:
# Method to concat all the reports files for each country in a single dataframe
def concat_files(path,date_str):
    the_files = glob.glob(path +"/*.csv")
    data_frame = pd.DataFrame()
    for x in the_files:
        df_temp = pd.read_csv(x,parse_dates=[date_str])
        data_frame = pd.concat([data_frame,df_temp])
    return data_frame

# Method to add a Month column
def add_month(df):
    copy_df = df.copy()
    months = [calendar.month_name[x.month] for x in copy_df.Date]
    copy_df['Month'] = months
    return copy_df

# Method to calculate the average number of new cases per month
def calculate_avg_new_cases(df, totals_id='New Cases'):
    months = df['Month'].unique()
    avg_cases = np.zeros(len(months))
    for i in range(len(months)):
        temp = (df.loc[df['Month'] == months[i]]) # Select relevant month
        temp = temp.dropna()
        temp = temp[totals_id].values.astype(float) # Extract array of floats
        if(len(temp) < 2 ): # If less than two points, assign NaN
            avg_cases[i] = np.nan
        else: # Otherwise sum/number
            avg_cases[i] = temp.sum()/temp.shape[0]
        if avg_cases[i] < 0:
            raise ValueError('Value of an average is less than 0 during month:', months[i])
    return avg_cases  

# Method to calculate the average number of deaths per month
def calculate_avg_new_deaths(df, totals_id='Total Deaths'):
    months = df['Month'].unique()
    avg_cases = np.zeros(len(months))
    for i in range(len(months)):
        temp = (df.loc[df['Month'] == months[i]]) # Selecting relevant month
        temp = temp.dropna()
        temp = temp.reset_index()  
        if(len(temp) > 1):
            delta_t = temp['Date'].loc[len(temp)-1] - temp['Date'].loc[0] # calculate time-span of data
        temp = temp[totals_id].values.astype(float) # Extracting array of floats   
        if(len(temp) < 2): # If less than two points, assign NaN
            avg_cases[i] = np.nan
        elif(temp[len(temp)-1]-temp[0] > 0):   # if there is a difference between 1st/last time point
            avg_cases[i] = (temp[len(temp)-1]-temp[0])/delta_t.days
        else: 
            avg_cases[i] = (temp[len(temp)-2]-temp[0])/delta_t.days
        if avg_cases[i] < 0:
            raise ValueError('Value of an average is less than 0 during month:',months[i])
    return avg_cases   

# Method to handle Sierra Leon and Liberia data
# Todo: should ideally replace this with Pandas functions, so we get proper NaN handling
# and avoid the SettingWithCopyWarning
def handle_data(df,desc1,desc2,desc3,new_total):
    sl_1 = df[df['Description'] ==desc1]
    sl_2 = df[df['Description'] ==desc2]
    sl_3 = df[df['Description'] ==desc3]
    new_cases_registered =  [int(a) + int(b) + int(c) for a,b,c in zip(sl_1.Totals,sl_2.Totals,sl_3.Totals)]
    sl_1[new_total] = new_cases_registered
    sl_1 = sl_1[['Date',new_total]]
    return sl_1

In [5]:
# Create Guinea DF
guinea_folder = DATA_FOLDER + '/ebola/guinea_data/'
g_df = concat_files(guinea_folder,'Date')
print('Shape of Guinea DF:', g_df.shape)

# We can check all unique descriptions in order to choose what 
# to base our calculations on as well as to be sure not to miss anything.
# add print() to display these descriptions
g_df['Description'].unique()

# We can then see which descriptors appear to be the most complete,
# in this case it seems using one of the total deaths descriptors
# is the only sensible option, the new deaths data sets are much
# less complete. We'll take the confirmed+probables+suspects, so
# the end results are probably overestimating the deaths a bit
print(g_df.loc[g_df['Description']=='Total deaths of probables'].shape)
print(g_df.loc[g_df['Description']=='Total deaths of confirmed'].shape)
print(g_df.loc[g_df['Description']=='Total deaths (confirmed + probables + suspects)'].shape)
print(g_df.loc[g_df['Description']=='New deaths registered'].shape)
print(g_df.loc[g_df['Description']=='New deaths registered today'].shape)
print(g_df.loc[g_df['Description']=='New deaths registered today (probables)'].shape)
print(g_df.loc[g_df['Description']=='New deaths registered today (suspects)'].shape)
g_df.loc[g_df['Description']=='Total deaths (confirmed + probables + suspects)'][['Date','Description','Totals']]

# We can already see in this table that the value for
# 2014-08-26 seems wrong compared to its neighbours, however
# because of how we choose to calculate that average daily
# deaths this won't matter

NameError: name 'pd' is not defined

We finally get to calculate average daily new cases and deaths per month.
In this data, we assume that the *Total deaths (confirmed + probables + suspects)* row we used from the original data is a cumulative value of the deaths. Therefore we'll calculate the daily average per month of deaths by taking the difference at the beginning and end of a month and divide by the number of days the data points cover.

However, the case for the *Total new cases registered so far* column was not as clear. If we look at the output of the next cell, the values for August appear to increasee monotonically as a a cumulative variable would. While the values for September seem to fluctuate more. We therefore chose to interpret this column as *daily* new cases registered, as this is compatible with the fluctuations observed in September. The daily average per month of new cases is therefore calculated as the average of these data points.

In both cases we require at least two data points to calculate an average, otherwise we return a `NaN`.

In [ ]:
# We only keep the columns of interest
g_df = g_df[['Date', 'Description', 'Totals']]

# We select the rows that we need in order to calculate our values
# We'll create a new df with two columns 'Total Deaths' and 'New Cases'
g_new_cases = (g_df.loc[g_df['Description'] == 'Total new cases registered so far'])
g_new_cases = g_new_cases.rename(columns={'Totals':'New Cases'})
g_new_cases.drop('Description', axis=1, inplace=True)

g_total_deaths = (g_df.loc[g_df['Description'] == 'Total deaths (confirmed + probables + suspects)'])
g_total_deaths = g_total_deaths.rename(columns={'Totals':'Total Deaths'})
g_total_deaths.drop('Description', axis=1, inplace=True)

# Create a merged DataFrame with these values
g_df = pd.merge(g_total_deaths, g_new_cases, on='Date', how='outer')

# Add 'Month' and 'Country' columns to df
g_df = add_month(g_df) # Add a 'Month' column, parsed from the 'Date' column
g_df['Country'] = 'Guinea' # Add a 'Country' column, for later merging
g_df.sort_values('Date', inplace=True)
g_df.head(10)

# We see that the data from the 2014-08-26 is not consistant but since we only use
# the first and last value of each month to calculate the avg we ignore it.

In [ ]:
print('Daily average deaths per month:',calculate_avg_new_deaths(g_df))
print('Daily average new cases per month:',calculate_avg_new_cases(g_df))

In [ ]:
# Create the Sierra Leone DataFrame
sl_folder = DATA_FOLDER + '/ebola/sl_data/'
sl_df = concat_files(sl_folder,'date')

# We only keep the columns of interest 
sl_df = sl_df[['date','variable','National']]
sl_df.columns = ['Date' , 'Description' , 'Totals']
sl_df = sl_df.fillna(0)

# Creating dataframes for new cases and new deaths
sl_new_cases = handle_data(sl_df,'new_confirmed','new_probable','new_suspected','New Cases')
sl_new_cases = sl_new_cases.loc[sl_new_cases['New Cases'] > 0] # filtering out 0 cases
sl_new_deaths = handle_data(sl_df,'death_confirmed','death_probable','death_suspected','Total Deaths')
sl_new_deaths = sl_new_deaths[sl_new_deaths['Total Deaths'] > 0] # filtering out 0 cases

# Forming the Sierra Leon DataFrame
sl_df = pd.merge(sl_new_cases, sl_new_deaths, on='Date', how='outer')
sl_df['Country'] = 'Sierra Leone' 
sl_df.sort_values('Date', inplace=True)
sl_df = add_month(sl_df) # Add a 'Month' column, parsed from the 'Date' column
sl_df.head()

In [ ]:
# Calculating the daily average per month of new cases and deaths for Sierra Leon

print('Daily average deaths per month:',calculate_avg_new_deaths(sl_df))
print('Daily average new cases per month:',calculate_avg_new_cases(sl_df))

In [ ]:
# Create the Liberia DataFrame
lib_folder = DATA_FOLDER + '/ebola/liberia_data/'
lib_df = concat_files(lib_folder,'Date')

# We only keep the columns of interest 
lib_df = lib_df[['Date','Variable','National']]
lib_df.columns = ['Date' , 'Description' , 'Totals']
lib_df = lib_df.fillna(0)
lib_df.head(10)

In [ ]:
# Creating dataframes for new cases and new deaths
lib_new_cases = handle_data(lib_df,'New Case/s (Suspected)','New Case/s (Probable)','New Case/s (Probable)','New Cases')
lib_new_cases = lib_new_cases.loc[lib_new_cases['New Cases'] > 0]
lib_new_deaths = handle_data(lib_df,'Total death/s in confirmed cases','Total death/s in probable cases','Total death/s in suspected cases','Total Deaths')
lib_new_deaths = lib_new_deaths.loc[lib_new_deaths['Total Deaths'] > 0]

In [ ]:
# Forming the Liberia DataFrame
lib_df = pd.merge(lib_new_cases, lib_new_deaths, on='Date', how='outer')
lib_df['Country'] = 'Liberia'
lib_df.sort_values('Date', inplace=True)
lib_df = add_month(lib_df) # Add a 'Month' column, parsed from the 'Date' column

In [ ]:
print('Daily average deaths per month:',calculate_avg_new_deaths(lib_df))
print('Daily average new cases per month:',calculate_avg_new_cases(lib_df))

In [ ]:
#Forming the Final DataFrame contaning the relevant information for the 3 different counties

df_merged = pd.concat([g_df,sl_df,lib_df])
df_merged.head()
df_merged[df_merged['Country'] == 'Guinea']
print('Values for Guinea')
print('Daily average deaths per month:',calculate_avg_new_deaths(df_merged[df_merged['Country'] == 'Guinea']))
print('Daily average new cases per month:',calculate_avg_new_cases(df_merged[df_merged['Country'] == 'Guinea']))
print('\nValues for Sierra Leone')
print('Daily average deaths per month:',calculate_avg_new_deaths(df_merged[df_merged['Country'] == 'Sierra Leone']))
print('Daily average new cases per month:',calculate_avg_new_cases(df_merged[df_merged['Country'] == 'Sierra Leone']))
print('\nValues for Liberia')
print('Daily average deaths per month:',calculate_avg_new_deaths(df_merged[df_merged['Country'] == 'Liberia']))
print('Daily average new cases per month:',calculate_avg_new_cases(df_merged[df_merged['Country'] == 'Liberia']))

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

First we are going to concatenate all excel files into one `DataFrame`. We'll make sure we have a unique index by resetting it to a temporary 0-x numerical index. We're going to be using the variable `xl_lengths` to keep track of the size of the different tables that are concatenated, and the variable `count` to double check that we import all the excel rows and don't miss something.

In [ ]:
mb_data = pd.DataFrame(columns=['raw','counts'])
nb_excels = 9
xl_lengths = np.zeros(nb_excels, dtype=int)
counts = 0

# Concat all excel files
for i in range(nb_excels):
    mb_new = pd.read_excel(DATA_FOLDER+"/microbiome/MID"+str(i+1)+".xls", header=None, names=['raw','counts'])
    mb_new.fillna('unknown', inplace=True)
    xl_lengths[i] = np.size(mb_new.index)
    counts += mb_new['raw'].size
    mb_data = pd.concat([mb_data, mb_new], axis=0, join='outer')

# Reset indices
mb_data = mb_data.reset_index() # creates a new 'index' column
mb_data = mb_data.drop('index', axis=1) # dropping this new column

# We visually double-check we've imported all the information
# by comparing the counts variable to the size of the df
print('Nb. of data points:', counts)
print('Data frame shape', mb_data.shape)
if counts == mb_data.shape[0]:
    print('Great! Looks like we have imported all the data')
mb_data.head()

We read in the metadata and fill in the `NaN` values with `unknown`, as asked. 

In [ ]:
mb_metadata = pd.read_excel(DATA_FOLDER+"/microbiome/metadata.xls")
mb_metadata.fillna('unknown', inplace=True)
mb_metadata.head(3)

We now initialise the new columns that will be filled in with the metadata information

In [ ]:
mb_data = pd.concat([mb_data, pd.DataFrame(columns=mb_metadata.columns)], axis=1)
print('Data frame shape:',mb_data.shape)
mb_data.head()

We now add the metadata to the corresponding slices of the overall `DataFrame` using our previous `xl_length` variable to keep track of when one imported tables end and another begins.

In [ ]:
idx = 0
for j in range(np.size(xl_lengths)):
    if j >= 1:
        idx = sum(xl_lengths[0:j])
    for i in mb_metadata.columns:
        mb_data[i][idx:idx+xl_lengths[j]+1] = mb_metadata[i][j]

print(xl_lengths)
print('Final DataFrame index is unique', mb_data.index.is_unique)
mb_data.head()

We now create a meaningful index using all the metadata that we imported and joined together, as well as the name of the microbe being sampled. We'll also check that the index is unique.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_excel(DATA_FOLDER+'/titanic.xls', sheetname='titanic', header=0)

## For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

### Task 3.1 

In [ ]:
# Question: Describe the type and the value range of each attribute. 

# We assume that we can describe the value range of each attribute
# only if it's a numerical type

types = df.dtypes
value_range = df._get_numeric_data().apply(lambda x: (x.min(), x.max()))

types.index.name = 'Attribute'
types.columns = 'Type'

value_range.index.name = 'Attribute'
value_range.columns = 'Range'

res = pd.concat(dict(Range=value_range, Type=types), axis=1).fillna('-')
res



In [ ]:
# Question: Indicate and transform the attributes that can be Categorical.

# By visual inspection we can argue that the following columns could be
# categorical : 'pclass', 'survived', 'sex', 'sibsp', 'parch', 'embarked'
#
# We assumed that for an attribute to be categorical it should have 
# <= 10 distinct elements. (NaN is not considered as a distinct value)



categories = pd.Series([v for v in df.columns if len(df[v].unique()) < 10 ])

print('Attributes to be made Categorical:\n\n', categories)
for i in categories:
    df[i] = df[i].astype('category')

### Task 3.2

In [ ]:
# Question: Plot a histogram for the travel class attribute
%matplotlib inline


hist_plot = df.pclass.value_counts().plot(kind='bar')
hist_plot.set_title('Travel Class')
hist_plot.set_xlabel('Class')
hist_plot.set_ylabel('Passengers')
np.sum(df.pclass.value_counts())

In [ ]:
# Question: Plot a histogram for the embarkation port attribute
%matplotlib inline

nb_ports = df['embarked'].describe()['unique']
df.embarked.unique()

hist_plot = df.embarked.value_counts().plot(kind='bar')
hist_plot.set_title('Embarkation port')
hist_plot.set_xlabel('Port')
hist_plot.set_ylabel('Passengers')
sum(df.embarked.value_counts())

In [ ]:
# Question: Plot a histogram for the sex attribute
%matplotlib inline

hist_plot = df.sex.value_counts().plot(kind='bar')
hist_plot.set_title('Sex')
hist_plot.set_xlabel('Sex')
hist_plot.set_ylabel('Passengers')
np.sum(df.sex.value_counts())

In [ ]:
# Question: Plot a histogram for the age attribute


nb_bins = np.ceil(df.age.max()/10).astype('int')
hist_plot = df.age.hist(bins=nb_bins, grid=False, xlabelsize=11, ylabelsize=11, figsize=(10, 6))
hist_plot.set_title('Age')
hist_plot.set_xlabel('Ages')
hist_plot.set_ylabel('Passengers')

### Taks 3.3

In [ ]:
# Question: Calculate the proportion of passengers by cabin floor. 
# Present your results in a pie chart.

# We assumed that the information on the cabin floor is available 
# in the column 'cabin' where the letter indicates the floor.
# the number that follows indicates a specific cabin on that floor.

# Unfortunately, this column is also filled with NaN's values.
# For this statistic we won't try guessing the cabin floor for 
# passenger where the data is not provided. Our analysis relies
# only on values that are already in the dataset.

tot = df.cabin.describe()['count']

temp = df.cabin.dropna(axis=0).apply(lambda x: str(x)[:1])

D = {}
for v in temp:
    if not v in D:
        D[v] = 1
    else:
        D[v] += 1


labels = list(D.keys())
values = list(D.values())


plt.pie(values, labels=labels)
plt.show()

### Task 3.4

In [ ]:
# Question: For each travel class, calculate the proportion of the
# passengers that survived. Present your results in pie charts.

# The pie chart represents only passengers that survived. Each region
# on the chart is the proportion of the passengers in the class
# that survived among all survivors.


tot = df.survived.describe()['count']

class_tot = (df.groupby('pclass').survived.describe())['count']
pclass_survived = df.groupby(['pclass', 'survived']).survived.describe()
pclass_survived = (pclass_survived[pclass_survived['top'] == 1])['freq']
pclass_survived = pclass_survived.reset_index('survived').drop('survived', axis=1)
stats = pd.concat([class_tot, pclass_survived], axis=1)
prop = (stats['freq'] / stats['count'])*100

l = df.set_index('pclass').index.categories
plt.pie(list(prop), labels=list(l))
plt.show()



### Task 3.5

In [ ]:
# Question: Calculate the proportion of the passengers that survived 
# by travel class and sex. Present your results in a single histogram.

class_tot = (df.groupby(['pclass', 'sex']).survived.describe())['count']
survived = df.groupby(['pclass', 'sex', 'survived']).survived.describe()
survived = (survived[survived['top'] == 1])['freq']
survived = survived.reset_index('survived').drop('survived', axis=1)
stats = pd.concat([class_tot, survived], axis=1)
prop = (stats['freq'] / stats['count'])*100

prop.unstack(level=1).plot(kind='bar', subplots=False)

### Task 3.6

In [ ]:
# Question: Create 2 equally populated age categories and calculate 
# survival proportions by age category, travel class and sex. Present
# your results in a DataFrame with unique index.

# Our analysis is based only on passengers that have a valid 'age'
# field. The field is considered valid only if the value is not a NaN.
# For this statistic we won't try guessing the passenger age when 
# the data is not provided.

# We create our 2 equally populated categories as follow:
# 1. We order the passengers by age in ascending order
# 2. We drop from the table passengers with invalid 'age' value
# 3. We separate the resulting dataframe in two with the cut
#    point being half way down the table

# The stats are then computed for each age category separately before 
# concatenating both resulting tables

# 1.
df.sort_values(['age'], axis=0, inplace=True)
# 2.
df.age.dropna(axis=0, inplace=True)
# 3.
mid = int(np.floor(len(df)/2))
cat1 = df[:mid]
cat2 = df[mid:]


In [ ]:
# Survival proportions by age category, travel class and sex
# Age Category #1

class_tot = (cat1.groupby(['age', 'pclass', 'sex']).survived.describe())['count']
survived = cat1.groupby(['age', 'pclass', 'sex', 'survived']).survived.describe()
survived = (survived[survived['top'] == 1])['freq']
survived = survived.reset_index('survived').drop('survived', axis=1)
stats = pd.concat([class_tot, survived], axis=1).fillna(0)
stats.columns = ['total', 'survivors']

s1 = stats.groupby(['pclass', 'sex']).sum()

s1['age'] = 'Younger'
s1.set_index('age', append=True, inplace=True)
s1 = s1.reorder_levels(['age', 'pclass', 'sex'])
s1

In [ ]:
# Survival proportions by age category, travel class and sex
# Age Category #2

class_tot = (cat2.groupby(['age', 'pclass', 'sex']).survived.describe())['count']
survived = cat2.groupby(['age', 'pclass', 'sex', 'survived']).survived.describe()
survived = (survived[survived['top'] == 1])['freq']
survived = survived.reset_index('survived').drop('survived', axis=1)
stats = pd.concat([class_tot, survived], axis=1).fillna(0)
stats.columns = ['total', 'survivors']

s2 = stats.groupby(['pclass', 'sex']).sum()

s2['age'] = 'Older'
s2.set_index('age', append=True, inplace=True)
s2 = s2.reorder_levels(['age', 'pclass', 'sex'])
s2

In [ ]:
# Survival proportions by age category, travel class and sex
df = pd.concat([s1, s2], axis=0)
df['proportion'] = (df['survivors'] / df['total'])*100

df

### Task 3.5

In [ ]:
# Question: Calculate the proportion of the passengers that survived 
# by travel class and sex. Present your results in a single histogram.

class_tot = (df.groupby(['pclass', 'sex']).survived.describe())['count']
survived = df.groupby(['pclass', 'sex', 'survived']).survived.describe()
survived = (survived[survived['top'] == 1])['freq']
survived = survived.reset_index('survived').drop('survived', axis=1)
stats = pd.concat([class_tot, survived], axis=1)
prop = (stats['freq'] / stats['count'])*100

prop.unstack(level=1).plot(kind='bar', subplots=False)

### Task 3.6

In [ ]:
# Question: Create 2 equally populated age categories and calculate 
# survival proportions by age category, travel class and sex. Present
# your results in a DataFrame with unique index.

# Our analysis is based only on passengers that have a valid 'age'
# field. The field is considered valid only if the value is not a NaN.
# For this statistic we won't try guessing the passenger age when 
# the data is not provided.

# We create our 2 equally populated categories as follow:
# 1. We order the passengers by age in ascending order
# 2. We drop from the table passengers with invalid 'age' value
# 3. We separate the resulting dataframe in two with the cut
#    point being half way down the table

# The stats are then computed for each age category separately before 
# concatenating both resulting tables

# 1.
df.sort_values(['age'], axis=0, inplace=True)
# 2.
df.age.dropna(axis=0, inplace=True)
# 3.
mid = int(np.floor(len(df)/2))
cat1 = df[:mid]
cat2 = df[mid:]


In [ ]:
# Survival proportions by age category, travel class and sex
# Age Category #1

class_tot = (cat1.groupby(['age', 'pclass', 'sex']).survived.describe())['count']
survived = cat1.groupby(['age', 'pclass', 'sex', 'survived']).survived.describe()
survived = (survived[survived['top'] == 1])['freq']
survived = survived.reset_index('survived').drop('survived', axis=1)
stats = pd.concat([class_tot, survived], axis=1).fillna(0)
stats.columns = ['total', 'survivors']

s1 = stats.groupby(['pclass', 'sex']).sum()

s1['age'] = 'Younger'
s1.set_index('age', append=True, inplace=True)
s1 = s1.reorder_levels(['age', 'pclass', 'sex'])
s1

In [ ]:
# Survival proportions by age category, travel class and sex
# Age Category #2

class_tot = (cat2.groupby(['age', 'pclass', 'sex']).survived.describe())['count']
survived = cat2.groupby(['age', 'pclass', 'sex', 'survived']).survived.describe()
survived = (survived[survived['top'] == 1])['freq']
survived = survived.reset_index('survived').drop('survived', axis=1)
stats = pd.concat([class_tot, survived], axis=1).fillna(0)
stats.columns = ['total', 'survivors']

s2 = stats.groupby(['pclass', 'sex']).sum()

s2['age'] = 'Older'
s2.set_index('age', append=True, inplace=True)
s2 = s2.reorder_levels(['age', 'pclass', 'sex'])
s2

In [ ]:
# Survival proportions by age category, travel class and sex
df = pd.concat([s1, s2], axis=0)
df['proportion'] = (df['survivors'] / df['total'])*100

df